In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/datamaestro2020/sample_submission.csv
/kaggle/input/datamaestro2020/astro_train.csv
/kaggle/input/datamaestro2020/astro_test.csv


In [ ]:
import numpy as np
import keras as K
import tensorflow as tf
import pandas as pd
import math

import numpy as np
import pandas as pd
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

import xgboost as xgb
from sklearn.model_selection import train_test_split
seed = 78
test_size = 0.2
from sklearn.metrics import accuracy_score

from numpy import loadtxt
from xgboost import XGBClassifier

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers
import keras as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
seed = 78
test_size = 0.2
import os

Using TensorFlow backend.


In [ ]:
train = pd.read_csv("/kaggle/input/datamaestro2020/astro_train.csv")
test = pd.read_csv("/kaggle/input/datamaestro2020/astro_test.csv")
sample = pd.read_csv("/kaggle/input/datamaestro2020/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
del train["id"]
del test["id"]
Y = train["class"]
del train["class"]

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train["run"] = (train["run"] - train["run"].min())/(train["run"].max() - train["run"].min())
#test["run"] = (test["run"] - test["run"].min())/(test["run"].max() - test["run"].min())
test["run"] = test["run"] / 752.0

train["field"] = (train["field"] - train["field"].min())/(train["field"].max() - train["field"].min())
test["field"] = (test["field"] - test["field"].min())/(test["field"].max() - test["field"].min())

train["obj"] = (train["obj"] - train["obj"].min())/(train["obj"].max() - train["obj"].min())
test["obj"] = (test["obj"] - test["obj"].min())/(test["obj"].max() - test["obj"].min())

train["#ra"] = (train["#ra"] - train["#ra"].min())/(train["#ra"].max() - train["#ra"].min())
test["#ra"] = (test["#ra"] - test["#ra"].min())/(test["#ra"].max() - test["#ra"].min())

train["dec"] = np.absolute(train["dec"])
test["dec"] = np.absolute(test["dec"])

#(v - v.min()) / (v.max() - v.min())

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
del train["rerun"]#only has one value that is 40
del test["rerun"] #40

del train["skyVersion"]#only has one value that is 1
del test["skyVersion"] #1

del train["run"]#two values 745 and 752
del test["run"]#752

del train["camCol"]#three values 1, 2, 3
del test["camCol"]#2

### values do not change

In [ ]:
train.info()

In [ ]:
print(np.shape(train))
print(np.shape(test))

In [ ]:
print(train)

In [ ]:
#from keras.utils import to_categorical

#Y = to_categorical(Y, num_classes = 3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, Y, test_size=0.2, random_state=2)

In [ ]:
from sklearn.model_selection import StratifiedKFold
n_fold = 4
folds = StratifiedKFold(n_splits=n_fold, shuffle=True)

In [ ]:
def randomforest_evaluate(**params):
    
    params['n_estimators'] = int(round(params['n_estimators'],0))
    params['min_samples_split'] = int(round(params['min_samples_split'],0))
    params['min_samples_leaf'] = int(round(params['min_samples_leaf'],0))
    params['bootstrap'] = int(round(params['bootstrap'],0))
        
    test_pred_proba = np.zeros((train.shape[0], 9))
    
    for n_fold, (train_idx, test_idx) in enumerate(folds.split(train, Y)):
        X_train, X_test = train.iloc[train_idx], train.iloc[test_idx]
        y_train, y_test = Y[train_idx], Y[test_idx]
        
        model = RandomForestClassifier(**params,n_jobs=-1)
        model.fit(X_train, y_train)

        y_pred_test = model.predict_proba(X_test)

        #test_pred_proba[test_idx] = y_pred_test

    return accuracy_score(y_test, y_pred_test.argmax(1))

In [ ]:
rf_param_grid = {
                 'max_depth' : (8,100),
                 'n_estimators': (50,2000),
                 'min_samples_split': (2,10),
                 'min_samples_leaf': (2, 10),
                 'bootstrap': (True, False),
                 }

rf_b_o = BayesianOptimization(randomforest_evaluate, rf_param_grid)
rf_b_o.maximize(init_points=5, n_iter=20)


In [ ]:
for i,n in rf_b_o.max["params"].items():
    print(i,int(round(n)))

In [ ]:
model=RandomForestClassifier(n_estimators=1044,max_depth=100,min_samples_leaf=2,min_samples_split=6)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(new_test_data)

In [ ]:
#categorical_features = ['dered_i','extinction_r','field', 'obj', 'photoz','#ra',  'dec']

In [ ]:
'''def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['auc-mean'][-1]
'''

In [ ]:
#import lightgbm

In [ ]:
'''lgbBO = BayesianOptimization(lgb_eval, {'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                         'min_samples_split': [2, 5, 10],
                                         'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
                                                })
'''


In [ ]:
#lgbBO.maximize(n_iter=10, init_points=2)

In [ ]:
#lgbBO.max

In [ ]:
#lgbBO.res[0]

In [ ]:
#import lightgbm as lgb
#d_train = lgb.Dataset(X_train, label = y_train)

In [ ]:
'''params = {}
params['lambda_l1'] = 0.03882575004510888
params['lambda_l2'] = 0.00907445519625818
params['min_child_samples'] = 5594.252982199325
params['num_leaves'] = 39
params['min_data_in_leaf'] = 1665
params['max_depth'] = 7
'''

In [ ]:
#clf = lgb.train(params, d_train, 100)

In [ ]:
'''y_pred = clf.predict(X_test)

predictions = [np.round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print(accuracy)
'''

In [ ]:
#random_forest = RandomForestClassifier(n_estimators=100,oob_score=True,max_features=5)

#random_forest.fit(X_train, y_train)

#y_pred = random_forest.predict(X_test)

#random_forest.score(X_train, y_train)

In [ ]:
#random_forest.score(X_test, y_test)

In [ ]:
#random_forest.get_params,random_forest.feature_importances_

In [ ]:
'''model = XGBClassifier(silent=False,
                      scale_pos_weight=1,
                      learning_rate=0.03,  
                      colsample_bytree = 0.4,
                      subsample = 1,
                      objective='multi:softmax', 
                      n_estimators=700, 
                      reg_alpha = 0.3,
                      max_depth=5, 
                      gamma=1,
                      num_class = 3)

model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
#print(np.shape(predictions))

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))'''

In [ ]:
'''model = Sequential()
model.add(Dense(activation="relu", input_dim=47, units=38, kernel_initializer="uniform"))
#model.add(Dropout(0.02))

model.add(Dense(activation="relu", units= 28, kernel_initializer="uniform"))
#model.add(Dropout(0.02))

#model.add(Dense(activation="relu", units=18, kernel_initializer="uniform"))
#model.add(Dropout(0.2))

model.add(Dense(activation="relu", units=16, kernel_initializer="uniform"))
#model.add(Dropout(0.02))

#model.add(Dense(activation="relu", units=12, kernel_initializer="uniform"))
#model.add(Dropout(0.2))

model.add(Dense(activation="relu", units=6, kernel_initializer="uniform"))
#model.add(Dropout(0.02))


model.add(Dense(activation="softmax", units=3, kernel_initializer="uniform"))
#we use dropout when model is overfitting so don't use dropout
'''

In [ ]:
'''K.optimizers.Adamax(learning_rate=0.0002, beta_1=0.9, beta_2=0.999)

model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['accuracy'])

from keras.callbacks import ReduceLROnPlateau
#learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1,  factor=0.5,  min_lr=0.00001)

model.summary()
'''

In [ ]:
#model.fit(X_train, y_train, epochs=15, batch_size=32) #epochs = 500 for 70%

In [ ]:
#predictions = model.predict(test)
#predict_class = np.argmax(predictions, axis=1)
#predict_class = predict_class.tolist()


#sample_submission = pd.read_csv("/kaggle/input/datamaestro2020/sample_submission.csv")

#submission_df = pd.DataFrame(columns=['id', 'class'])
#submission_df['id'] = sample_submission['id']
#submission_df['class'] = predict_class
#submission_df.to_csv('submissionsfinal.csv', header=True, index=False)
#submission_df.head(10)

In [ ]:
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)